In [ ]:
import requests
import csv
import time
import os
from dotenv import load_dotenv


load_dotenv()  # Load environment variables from .env file

# ==== SETTINGS ====
API_KEY = os.getenv("weather_api_key")   # Replace with your OpenWeatherMap API key
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
OUTPUT_FILE = "../scraped_data/nigeria_weather.csv"

# Example list of Nigerian cities (You can expand this list or use a full CSV)
cities = [
    "Lagos", "Abuja", "Kano", "Ibadan", "Port Harcourt", "Benin City",
    "Maiduguri", "Zaria", "Aba", "Jos", "Ilorin", "Enugu", "Oyo",
    "Abeokuta", "Onitsha", "Warri", "Calabar", "Sokoto", "Uyo", "Kaduna"
]

# ==== FUNCTION TO FETCH WEATHER ====
def get_weather(city):
    params = {
        "q": f"{city},NG",
        "appid": API_KEY,
        "units": "metric"
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return {
            "city": city,
            "lat": data["coord"]["lat"],
            "lon": data["coord"]["lon"],
            "weather": data["weather"][0]["main"],
            "description": data["weather"][0]["description"],
            "temperature_C": data["main"]["temp"],
            "humidity_%": data["main"]["humidity"],
            "wind_speed_mps": data["wind"]["speed"],
            "timestamp": data["dt"]
        }
    else:
        print(f"Failed to get data for {city}: {response.status_code}")
        return None

# ==== MAIN SCRIPT ====
if __name__ == "__main__":
    all_data = []
    
    for city in cities:
        weather_data = get_weather(city)
        if weather_data:
            all_data.append(weather_data)
        time.sleep(1)  # Respect API rate limit (60 calls/minute for free plan)
    
    # Save to CSV
    with open(OUTPUT_FILE, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=all_data[0].keys())
        writer.writeheader()
        writer.writerows(all_data)
    
    print(f"Weather data saved to {OUTPUT_FILE}")


Weather data saved to nigeria_weather.csv
